In [ ]:
# TOC
# Preparation
# (1) Logistic Regression
# (2) Decision Trees
# (3) Naive Bayes

Preparation

In [45]:
import numpy as np
import pandas as pd

In [46]:
# data from 2004 - 2021 (excluding the year 2020)
Outfield = pd.read_csv('outfield_data.csv')
Outfield = Outfield.loc[Outfield.year != 2020, ]

In [47]:
# excluding columns year, team, games, games started, and completed games
MLOutfield = Outfield.drop(['year', 'Tm', 'G', 'GS', 'CG'], axis = 1)

In [48]:
MLOutfield.head()

,#Fld,RA/G,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rtz,Rof,Rdrs,Rdrs/yr,Rpm,Rof.1,Rgood,Playoff
0,11,4.53,4363.0,1161,1110,32,19,6,0.984,-5,-1,-11,6,-17,-4,-26,7,2,1
1,13,5.55,4308.0,984,936,26,22,8,0.978,-26,-7,-22,-4,-1,0,13,-8,-5,0
2,10,4.12,4350.0,1100,1052,40,8,6,0.993,42,12,32,10,37,10,22,11,4,1
3,12,5.12,4366.0,1105,1073,24,8,5,0.993,18,5,17,1,-4,-1,6,-10,0,0
4,10,4.74,4354.0,1034,994,19,21,4,0.980,-23,-6,-17,-6,-28,-7,-19,-7,-1,1


In [49]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [50]:
X = MLOutfield.drop('Playoff', axis = 1) # independent
Y = MLOutfield.Playoff # dependent 

In [51]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y)

In [52]:
test_ml = MLOutfield.Playoff 

In [53]:
# Preparing current 2022 season:

In [54]:
# 2022 season stats - 8/5/22
Outfield22 = pd.read_csv('mlb22season.csv') 
Outfield22.head()

,Tm,#Fld,RA/G,G,GS,CG,Inn,Ch,PO,A,...,Fld%,Rtot,Rtot/yr,Rtz,Rof,Rdrs,Rdrs/yr,Rpm,Rof.1,Rgood
0,Arizona Diamondbacks,7,4.59,104,312,250,2749.0,778,748,20,...,0.987,8,4,11,-3,13,5,11,-1,3
1,Atlanta Braves,12,3.92,106,318,272,2848.0,636,618,9,...,0.986,-15,-6,-11,-4,-8,-3,-2,-3,-3
2,Baltimore Orioles,11,4.18,105,315,272,2789.0,708,677,23,...,0.989,-6,-3,-10,4,13,5,4,9,0
3,Boston Red Sox,9,4.68,107,321,269,2860.0,672,658,9,...,0.993,14,6,12,2,-4,-1,-2,0,-2
4,Chicago Cubs,11,4.95,104,312,252,2782.0,672,638,20,...,0.979,-19,-8,-22,3,-20,-8,-21,2,-1


In [55]:
Outfield22index = [col not in ['Tm', 'G', '#Fld', 'RA/G', 'CG', 'Fld%', 'Rdrs/yr', 'Rtot/yr'] for col in Outfield22.columns]
for col in Outfield22.columns[Outfield22index]:
    Outfield22[col] = Outfield22[col] * (162 / Outfield22.G) 

In [56]:
MLOutfield22 = Outfield22.drop(['Tm', 'G', 'GS', 'CG'], axis = 1) 
# same columns dropped as above except there's no year to begin with in this one
MLOutfield22.head()

,#Fld,RA/G,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rtz,Rof,Rdrs,Rdrs/yr,Rpm,Rof.1,Rgood
0,7,4.59,4282.096154,1211.884615,1165.153846,31.153846,15.576923,7.788462,0.987,12.461538,4,17.134615,-4.673077,20.250000,5,17.134615,-1.557692,4.673077
1,12,3.92,4352.603774,972.000000,944.490566,13.754717,13.754717,3.056604,0.986,-22.924528,-6,-16.811321,-6.113208,-12.226415,-3,-3.056604,-4.584906,-4.584906
2,11,4.18,4303.028571,1092.342857,1044.514286,35.485714,12.342857,4.628571,0.989,-9.257143,-3,-15.428571,6.171429,20.057143,5,6.171429,13.885714,0.000000
3,9,4.68,4330.093458,1017.420561,996.224299,13.626168,7.570093,3.028037,0.993,21.196262,6,18.168224,3.028037,-6.056075,-1,-3.028037,0.000000,-3.028037
4,11,4.95,4333.500000,1046.769231,993.807692,31.153846,21.807692,4.673077,0.979,-29.596154,-8,-34.269231,4.673077,-31.153846,-8,-32.711538,3.115385,-1.557692


In [57]:
MLOutfield22.shape # 30 teams, and playoff is the missing column

(30, 18)

(1) Logistic Regression model

Logistic regression is used to predict binary outcomes, and in this case would be "Yes" or "No" for if a team made it to the playoffs or not.

In [58]:
from sklearn.linear_model import LogisticRegression

In [59]:
logreg_model = LogisticRegression(max_iter = 10000)
logreg_model.fit(Xtrain, Ytrain)
predicted = logreg_model.predict(Xtest)
actual = Ytest

In [60]:
print(metrics.accuracy_score(predicted, actual)) # (1) accuracy score

0.78125


In [61]:
from collections import Counter

In [62]:
all_playoff_teams = Counter()
for iteration in range(50):
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y)   
    logreg_model = LogisticRegression(max_iter = 10000)
    logreg_model.fit(Xtrain, Ytrain)
    predictions_2022 = logreg_model.predict(MLOutfield22)
    DF_predictions_2022 = pd.DataFrame({'2022 Teams':Outfield22.Tm, 'Playoff Prediction':predictions_2022})
    playoff_teams_2022 = DF_predictions_2022.loc[DF_predictions_2022['Playoff Prediction'] == 1, '2022 Teams']
    all_playoff_teams.update(list(playoff_teams_2022))

In [63]:
pd.DataFrame({'Team':all_playoff_teams.keys(), 
              '# Replications In Playoffs':all_playoff_teams.values()}
            ).sort_values(by = '# Replications In Playoffs', ascending = False)

,Team,# Replications In Playoffs
2,Houston Astros,50
3,Los Angeles Dodgers,50
7,New York Yankees,50
9,Tampa Bay Rays,50
0,Atlanta Braves,49
8,St. Louis Cardinals,47
6,New York Mets,46
4,Miami Marlins,27
1,Chicago White Sox,21
11,San Diego Padres,8


(2) Decision Tree

In [65]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

In [66]:
dtc.fit(Xtrain, Ytrain)
Ypred = dtc.predict(Xtest)

In [67]:
predicted = dtc.predict(Xtest)
actual = Ytest
print(metrics.accuracy_score(predicted, actual))

0.703125


In [ ]:
season22pred = dtc.predict(MLOutfield22)
Playoff_Predictions = pd.DataFrame({'2022 Teams':Outfield22.Tm, 'Playoff Prediction':season22pred})

In [68]:
all_playoff_teams = Counter()
for iteration in range(100):
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y)   
    dtc = DecisionTreeClassifier()
    dtc.fit(Xtrain, Ytrain)
    predictions_2022 = dtc.predict(MLOutfield22)
    DF_predictions_2022 = pd.DataFrame({'2022 Teams':Outfield22.Tm, 'Playoff Prediction':predictions_2022})
    playoff_teams_2022 = DF_predictions_2022.loc[DF_predictions_2022['Playoff Prediction'] == 1, '2022 Teams']
    all_playoff_teams.update(list(playoff_teams_2022))

In [69]:
pd.DataFrame({'Team':all_playoff_teams.keys(), 
              '# Replications In Playoffs':all_playoff_teams.values()}
            ).sort_values(by = '# Replications In Playoffs', ascending = False)

,Team,# Replications In Playoffs
10,New York Yankees,91
9,New York Mets,82
14,Toronto Blue Jays,70
12,Tampa Bay Rays,66
3,Chicago White Sox,61
2,Boston Red Sox,59
13,Texas Rangers,57
5,Houston Astros,52
1,Atlanta Braves,44
19,St. Louis Cardinals,43
